## January 31, 2018: Linear algebra (part 2 of 2)

## Math 157: Intro to Mathematical Software
## UC San Diego, winter 2018


Administrivia:

- Please fill out the [week 4 questionnaire](https://docs.google.com/forms/d/e/1FAIpQLSeecb_KR5QHZWGxJcISzQuKG-F9fFJzfvE_K1-v7ctk2piT_Q/viewform?usp=sf_link).
- We are continuing to monitor the course waitlist. If you are on it and still want to join the course, please continue to keep up with lectures and homework, and watch your email for further instructions.
- Here is a more precise link for the [documentary screening next Monday](http://physicalsciences.ucsd.edu/programs/research-communications/mathcomm_eremenko_flier1.pdf) that I mentioned last time.


### Octave from Sage

So far we have seen Octave and Sage being accessed from a Jupyter notebook by switching kernels back and forth. However, this approach does not allow for any communication *between* the two kernels: all state is lost when one switches the kernel from Sage to Octave or vice versa.

It was asked whether there is a way to combine the functionality of the two kernels. The answer is yes, in one direction: Sage can call into Octave, but not vice versa. Some examples from the [reference manual](https://doc.sagemath.org/html/en/reference/interfaces/sage/interfaces/octave.html):

In [23]:
octave("besselj(1,2)")      # directly call a single Octave command

 0.576725

In [24]:
my_vector1 = octave('[1,5,7]') # create Python/Sage variables which point to Octave objects
my_vector2 = octave('[1;5;7]') 
my_vector1 * my_vector2 # Sage translates this operation into Octave for you.

 75

In [25]:
M33 = MatrixSpace(QQ,3,3)
A   = M33([1,2,3,4,5,6,7,8,0])
V3  = VectorSpace(QQ,3)
b   = V3([1,2,3])
octave.solve_linear_system(A,b)

[-0.333333, 0.666667, 0]

### Vector spaces

So far we have been talking about linear algebra solely in terms of matrices. However, Sage can also handle the language of vector spaces, as long as we are talking about subspaces of a "standard" vector space over some field.

In [26]:
V = VectorSpace(QQ, 5)
V

Vector space of dimension 5 over Rational Field

In [27]:
v = V([2,1,1,-1,1])
print(v)
print(type(v))


(2, 1, 1, -1, 1)
<type 'sage.modules.vector_rational_dense.Vector_rational_dense'>


In [30]:
type(vector([2,1,1,-1,1]))

<type 'sage.modules.vector_integer_dense.Vector_integer_dense'>

In [31]:
w = V([1,1,-1,0,2])
W = V.span([v,w]); W

Vector space of degree 5 and dimension 2 over Rational Field
Basis matrix:
[ 1  0  2 -1 -1]
[ 0  1 -3  1  3]

In [32]:
W.ambient_vector_space() == V

True

In [33]:
w2 = V([1,1,-1,0,2])
w3 = V([2,-2,0,5,-1])
W2 = V.span([w2, w3])

In [34]:
W.intersection(W2)

Vector space of degree 5 and dimension 1 over Rational Field
Basis matrix:
[ 1  1 -1  0  2]

In [35]:
M = Matrix(QQ, [[2,3,1,4],[0,1,-1,2]])
M.right_kernel()

Vector space of degree 4 and dimension 2 over Rational Field
Basis matrix:
[   1    0 -2/3 -1/3]
[   0    1 -1/3 -2/3]

In [43]:
M = Matrix([[2,0,0],[0,3,1],[-1,1,2]])
M.eigenspaces_right()

[
(2, Vector space of degree 3 and dimension 1 over Rational Field
User basis matrix:
[ 1  1 -1]),
(1.381966011250106?, Vector space of degree 3 and dimension 1 over Algebraic Field
User basis matrix:
[                  0                   1 -1.618033988749895?]),
(3.618033988749895?, Vector space of degree 3 and dimension 1 over Algebraic Field
User basis matrix:
[                 0                  1 0.618033988749895?])
]

In [44]:
M = Matrix([[0,0],[1,0]])
M.eigenspaces_right()

[
(0, Vector space of degree 2 and dimension 1 over Rational Field
User basis matrix:
[0 1])
]

In [37]:
A = matrix(QQ, 3, [1..9])
show(A)
V = A.image()
W = A.kernel()
print "image: "
print V
print "kernel: "
print W

image: 
Vector space of degree 3 and dimension 2 over Rational Field
Basis matrix:
[ 1  0 -1]
[ 0  1  2]
kernel: 
Vector space of degree 3 and dimension 1 over Rational Field
Basis matrix:
[ 1 -2  1]


In [38]:
A.restrict(V) # V = image

[-6 -6]
[18 21]

In [39]:
A.restrict(W)

[0]

In [40]:
show(factor(A.charpoly()))

In [41]:
show(factor(A.restrict(V).charpoly()))

In [42]:
M = span(QQ, [vector([1,0,0])]) 
M
A.restrict(M)   # A doesn't leave the subspace M invariant... so this will fail!

ArithmeticError: subspace is not invariant under matrix

### Sparse vs. dense matrices

In computational linear algebra, it is important to distinguish between *sparse* matrices, in which all but a few entries are zero, and *dense* matrices, where most of the entries are nonzero. This is not a formal mathematical distinction among matrices, but rather a distinction between two different classes of *algorithms* for handling matrices.

In [1]:
v = vector({2:4, 95:4, 1000000:3/4})
parent(v)

Sparse vector space of dimension 1000001 over Rational Field

In [47]:
v[3]

0

In [48]:
M = Matrix({(3,2): 27, (2,98): 71})
parent(M)

Full MatrixSpace of 4 by 99 sparse matrices over Integer Ring

In [49]:
print(M.is_sparse(), M.is_dense())

(True, False)


Note that in the previous examples, Sage tried to guess the dimension of the matrix. If it is guessing wrong, you may need to fix this by specifying dimensions.

In [2]:
M = Matrix(QQ, 1000001, 1000001, {(3,2): 27, (2,98): 71})
parent(M)

Full MatrixSpace of 1000001 by 1000001 sparse matrices over Rational Field

In [0]:
M2 = M*M # Do not try this with dense matrices of this size!

In [0]:
w = M*v # Don't try to print this answer either.

In [0]:
sum(i for i in w)

**Warning**: Not all operations make sense to do in the sparse model. For example, passing from a matrix to its inverse does not preserve sparsity in general. Many operations on sparse matrices involve an implicit coercion to a dense matrix.

### Operations on matrices

In [1]:
A = matrix(QQ, [[1,2],[3,4]])
A^3 - 2*A + 1

[ 36  50]
[ 75 111]

In [2]:
A+1

[2 2]
[3 5]

In [3]:
A = matrix(QQ, [[1,2],[3,4]])
f(x) = x^3 - 2*x + 1
f(A) # FAIL -- this should work, but doesn't

TypeError: no canonical coercion from Full MatrixSpace of 2 by 2 dense matrices over Rational Field to Callable function ring with argument x

In [4]:
R.<y> = PolynomialRing(QQ)
parent(y)

Univariate Polynomial Ring in y over Rational Field

In [5]:
A = matrix(QQ, [[1,2],[3,4]])
R.<y> = PolynomialRing(QQ)
f = y^3 - 2*y + 1
f(A)  # works

[ 36  50]
[ 75 111]

In [6]:
show(A.exp())   # compute sum A^k/k!
type(A.exp())

<type 'sage.matrix.matrix_symbolic_dense.Matrix_symbolic_dense'>

In [11]:
RR

Real Field with 53 bits of precision

In [12]:
RDF

Real Double Field

In [15]:
A.change_ring(RDF).exp()

[51.968956198705044  74.73656456700328]
[112.10484685050491 164.07380304920997]

In [19]:
print("A.inverse()")
print(A.inverse())
print("A^(-1)")
print(A^(-1))
print("~A")
print(~A)
print("A.transpose()")
print(A.transpose())
print("A.conjugate() -- boring since real")
print(A.conjugate())
print("A.antitranspose()")
print(A.antitranspose())
print("A.adjoint()")
print(A.adjoint())

A.inverse()
[  -2    1]
[ 3/2 -1/2]
A^(-1)
[  -2    1]
[ 3/2 -1/2]
~A
[  -2    1]
[ 3/2 -1/2]
A.transpose()
[1 3]
[2 4]
A.conjugate() -- boring since real
[1 2]
[3 4]
A.antitranspose()
[4 2]
[3 1]
A.adjoint()
[ 4 -2]
[-3  1]


### More on NumPy

In [20]:
import numpy as np

In [22]:
v = np.arange(15) 
v

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [23]:
a = v.reshape(3,5)
a

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

In [25]:
print(a.shape, a.ndim, a.dtype)

((3, 5), 2, dtype('int64'))


Notice that I typed `a.shape` rather than `a.shape()` as you may have expected if you have gotten accustomed to Python syntax. The point is that `shape` is not a *method* of a NumPy array but rather an *attribute*, essentially a private variable belonging to the array.

In [26]:
c = np.array([ [1,2], [3,4] ], dtype=complex) # The entry type can be forced at creation
c

array([[ 1.+0.j,  2.+0.j],
       [ 3.+0.j,  4.+0.j]])

The following is an example of a *vectorized operation*, where we apply a function to an array, which applies that function componentwise. NumPy is set up to do this sort of thing very efficiently (a la Fortran).

In [27]:
v = np.linspace(0, 2, 5); v

array([ 0. ,  0.5,  1. ,  1.5,  2. ])

In [28]:
w = np.sin(v); w

array([ 0.        ,  0.47942554,  0.84147098,  0.99749499,  0.90929743])

Let's test out the speed using the Python function `timeit`.

In [33]:
v = np.arange(0,1,1000000)
timeit('w = np.sin(v)')

625 loops, best of 3: 1.69 µs per loop


In [34]:
timeit('w = np.array([np.sin(x) for x in v])')

625 loops, best of 3: 3.69 µs per loop


Observe the difference:

In [35]:
print("Numpy:")
a = np.array([[1,2],[3,4]])
print(np.exp(a))

Numpy:
[[  2.71828183   7.3890561 ]
 [ 20.08553692  54.59815003]]


In [36]:
print("Sage:")
b = matrix(RDF,[[1,2],[3,4]]) # RDF = RealDoubleField()
print(exp(b))

Sage:
[51.968956198705044  74.73656456700328]
[112.10484685050491 164.07380304920997]


In [37]:
print("Numpy:")
print(a * a)
print("Sage:")
print(b * b)

Numpy:
[[ 1  4]
 [ 9 16]]
Sage:
[ 7.0 10.0]
[15.0 22.0]


This is just an arbitrary choice, made differently by mathematicians and engineers, so you have to
be aware of it.

In [38]:
np.exp(a)

array([[  2.71828183,   7.3890561 ],
       [ 20.08553692,  54.59815003]])

In [39]:
b.apply_map(exp)

[ 2.718281828459045   7.38905609893065]
[20.085536923187668 54.598150033144236]

In [40]:
b.apply_map(lambda x: x**2)

[ 1.0  4.0]
[ 9.0 16.0]

In [41]:
# and...
a.dot(a)

array([[ 7, 10],
       [15, 22]])

### Some fun

In [42]:
matrix_plot(random_matrix(RDF,50)^3, cmap='summer', colorbar=True)

In [0]:
matrix_plot(matrix(50,50,lambda i,j: randint(0,1)))

In [0]:
v = vector([1,2,3])
w = vector([0,2,5])

var('s, t')
g = parametric_plot3d(s*v + t*w, (s, -1,1), (t,-1,1), opacity=.5)

v = vector([1,-2,3])
w = vector([0,3,-5])
h = parametric_plot3d(s*v + t*w, (s, -1,1), (t,-1,1), color='red')

#i = parametric_plot3d(...., (s, -2, 2), ...)
#g+h+i
show(g+h)